https://www.kaggle.com/c/quora-insincere-questions-classification/data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

In [3]:
# path = Path('/Users/nikhil/workspace/fastaiv1/my_data/quora')

In [4]:
path = Path(Config.get_key('data_path')).expanduser()/'quora'
os.makedirs(path, exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/quora')

In [5]:
# ! /opt/anaconda3/bin/kaggle competitions download -c quora-insincere-questions-classification -f train.csv -p {path}
# ! /opt/anaconda3/bin/kaggle competitions download -c quora-insincere-questions-classification -f test.csv -p {path}
# ! unzip {path}/train.csv.zip -d {path}
# ! unzip {path}/test.csv.zip -d {path}
# ! chmod 775 -R {path}

In [6]:
df = pd.read_csv(path/'train.csv')

In [7]:
def label(row):
    return 'positive' if row.target else 'negative'

df['label'] = df.apply(lambda row: label(row), axis=1)
df = df.rename(columns = {"question_text": "text"})
df = df.drop(['qid', 'target'], axis=1)

In [14]:
import random
# assign true value with probability of 0.8
df['is_valid'] = df.apply(lambda x:0.8 > random.random(), axis=1)

#len(df) == len(df.qid.unique()) #1306122

In [15]:
len_df = int(len(df)*.8)
train_df, valid_df = df[:len_df], df[len_df:]
# for cpu local
train_df = train_df[:int(len(train_df)*.01)]
# valid_df = valid_df[:int(len(valid_df)*.01)]
train_df.head()

,text,label,is_valid
0,How did Quebec nationalists see their province...,negative,True
1,"Do you have an adopted dog, how would you enco...",negative,True
2,Why does velocity affect time? Does velocity a...,negative,True
3,How did Otto von Guericke used the Magdeburg h...,negative,True
4,Can I convert montra helicon D to a mountain b...,negative,True


In [16]:
train_df.to_csv(path/'texts.csv', index=False, columns=['label', 'text', 'is_valid'])
# valid_df.to_csv(path/'valid.csv', index=False, columns=['label', 'text', 'is_valid'])

In [21]:
# data_lm = TextDataBunch.from_csv(path, train='texts', valid='valid')
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [22]:
data_lm.label_from_func(lambda x:0)  

AttributeError: 'NumericalizedDataset' object has no attribute 'label_from_func'

In [18]:
data_lm.save()

Total time: 00:12
epoch  train_loss  valid_loss  accuracy
1      4.374722    3.419816    0.404745  (00:12)



In [19]:
learn.load('fit_head')

In [48]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7), wd=0.03, pct_start=0.25)

Total time: 02:36
epoch  train_loss  valid_loss  accuracy
1      3.421395    3.035594    0.452136  (00:16)
2      3.214228    2.924971    0.461280  (00:15)
3      3.086314    2.867447    0.468241  (00:15)
4      2.970393    2.827597    0.468482  (00:15)
5      2.868867    2.819221    0.471806  (00:15)
6      2.773415    2.801524    0.475083  (00:15)
7      2.696057    2.811234    0.472876  (00:15)
8      2.628194    2.819151    0.472208  (00:15)
9      2.579053    2.823037    0.471669  (00:15)
10     2.557436    2.820148    0.471985  (00:15)



In [49]:
learn.save('fine_tuned')
learn.save_encoder('fine_tuned_enc')

In [50]:
learn.freeze()

In [23]:
doc(TextFileList)

In [24]:
text_classifier_learner??